In [1]:
import json
import pandas as pd

In [2]:
f = open('bible.json')
data = json.load(f)

In [3]:
flattened_data = []

for book_key, book_value in data['books'].items():
    book_name = book_value.get('name', book_key)  # Assuming each book has a 'name'
    for chapter_key, chapter_value in book_value.get('chapters', {}).items():
        for verse_key, verse_value in chapter_value.get('verses', {}).items():
            for references_key, references_value in verse_value.get('references', {}).items():
                for reference in references_value:
                    row = {
                        'Book': book_name,
                        'Verse ID': verse_value.get('id', ''),
                        'References': reference.get('id', ''),
                    }
                    flattened_data.append(row)


In [4]:
df = pd.DataFrame(flattened_data)
split_cols = df['Verse ID'].str.split('.',expand=True)
df[['Book No.', 'Chapter No.','Verse No.']] = split_cols
split_cols = df['References'].str.split('.',expand=True)
df[['Ref Book No.', 'Ref Chapter No.','Ref Verse No.']] = split_cols
df

,Book,Verse ID,References,Book No.,Chapter No.,Verse No.,Ref Book No.,Ref Chapter No.,Ref Verse No.
0,Genesis,1.1.1,19.33.6,1,1,1,19,33,6
1,Genesis,1.1.1,19.33.9,1,1,1,19,33,9
2,Genesis,1.1.1,19.136.5,1,1,1,19,136,5
3,Genesis,1.1.1,43.1.1,1,1,1,43,1,1
4,Genesis,1.1.1,51.1.16,1,1,1,51,1,16
...,...,...,...,...,...,...,...,...,...
268720,Revelation,66.22.21,50.1.2,66,22,21,50,1,2
268721,Revelation,66.22.21,50.4.23,66,22,21,50,4,23
268722,Revelation,66.22.21,51.1.2,66,22,21,51,1,2
268723,Revelation,66.22.21,52.1.1,66,22,21,52,1,1


In [5]:
df.to_csv('export.csv')

In [6]:
import polars as pl

pldf = pl.read_csv('export.csv')

In [7]:
no_null = pldf.drop_nulls()

In [8]:
book_index = no_null.select(
                    'Book','Book No.'
                ).unique().sort(
                    'Book No.'
                ).with_columns(
                    pl.when(
                    pl.col('Book No.') < 40
                    ).then(pl.lit('OT')).otherwise(pl.lit('NT')).alias('Testament')
                )
                
# book_index.filter(pl.col('Book No.') == 23)

In [9]:
no_null.group_by('References').agg(pl.count()).sort('count', descending=True).head()

References,count
str,u32
"""23.9.6""",172
"""47.5.21""",168
"""23.3.11""",166
"""24.25.9""",164
"""43.1.14""",150


In [13]:
no_null.filter(pl.col('References') == '23.9.6').drop('').write_csv('references_to_isa_9_6.csv')

In [142]:
no_null.group_by('Ref Book No.').agg(
    pl.count()
    ).with_columns(
        (
        (
            pl.col('count')
            /no_null.select(pl.count()
            ).item()
        )*100
    ).alias(
        'ratio'
    )
    ).join(
        book_index, left_on='Ref Book No.',right_on='Book No.'
    ).sort(
        'count', descending=True
    ).head()

Ref Book No.,count,ratio,Book,Testament
i64,u32,f64,str,str
19,26819,10.772197,"""Psalms""","""OT"""
23,20034,8.046914,"""Isaiah""","""OT"""
24,13561,5.44695,"""Jeremiah""","""OT"""
40,9908,3.979676,"""Matthew""","""NT"""
5,8711,3.498885,"""Deuteronomy""","""OT"""


In [146]:
ref_by_book = \
no_null.group_by('Book','Ref Book No.').agg(
    pl.count()
    ).join(
        book_index, left_on='Ref Book No.',right_on='Book No.'
    ).join(
        (no_null.group_by('Book').agg(
    pl.count()
        )),on='Book'
    ).with_columns(
        (
        (
            pl.col('count')
            /pl.col('count_right')
        )*100
    ).alias(
        'percentage_of_book'
    )).filter(
        pl.col('Book') != pl.col('Book_right')
    ).sort(
        ['Book','percentage_of_book'], descending=[False,True]
    ).select(
     'Book','Book_right','count','count_right','percentage_of_book'   
    ).rename({'count_right':'total','Book_right':'Reference Book'})
    
ref_by_book.write_csv('ref_by_book.csv')

Make a stacked bar chart with each Books reference books